# **Team - Nexus Interrogators**

# **Subtask 2 V2: Human-AI Collaborative Text Classification**

- Here we build on our intitial finetuning approach.

- We do data augmentation on our dataset, and finetune the model on this updated dataset.

- Results can be seen below in the notebook.

- To run it, you can run all cells of this notebook individually.

- External script `baseline_st2.py` used below. Make sure it is in the same directory as this notebook.

## **Installing Dependencies**

In [ ]:
!pip install transformers datasets torch scikit-learn gdown evaluate
!pip install nlpaug sentencepiece sacremoses

## **Importing Libraries**

In [2]:
import json
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
from datasets import Dataset, concatenate_datasets
import numpy as np
import os
import torch
import evaluate

import nlpaug.augmenter.word as naw
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.sentence as nas
import nltk
import sentencepiece
from collections import Counter
from datasets import load_dataset

## **Loading Data**

In [ ]:
target_dir = "/kaggle/working/st2data"
os.makedirs(target_dir, exist_ok=True)
!gdown "https://drive.google.com/uc?id=1u5C4o_fmjL5nQ_RtgLDShuG97Ix6_KGK" -O "/kaggle/working/st2data/train.jsonl"
!gdown "https://drive.google.com/uc?id=1rNQTkhkVG9nzcT97Nk_WyJd80ZaacT0-" -O "/kaggle/working/st2data/val.jsonl"

In [ ]:
# Function to load and extract required fields
def load_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            entry = json.loads(line)
            data.append({"text": entry["text"], "label": entry["label"]})
    return pd.DataFrame(data)  # Convert to DataFrame

# Load training and validation data into DataFrames
train_df = load_data("st2data/train.jsonl")
val_df = load_data("st2data/val.jsonl")

# Display first few rows
print("Training Data:")
print(train_df.head())

print("\nValidation Data:")
print(val_df.head())

# text, language, label, source_dataset, model, label_text

Training Data:
                                                text  label
0  Have you ever had to wait for something for a ...      4
1  But now, things were not so simple._SEP_The gi...      3
2  Dear Editor,  I am writing to express my opini...      4
3  Humans once wielded formidable magical power. ...      4
4  Here is a way that I had to be patient, and we...      4

Validation Data:
                                                text  label
0  In a brief speech with a main theme of comprom...      0
1  HOW TO MAKE: Fall Squash Vegetarian Brioche\nI...      0
2  The paper models the relation extraction probl...      3
3  This paper considers the problem of model-base...      3
4  Paper Summary:\n\nAuthors investigate identity...      3


In [5]:
# converting the data to the hugging face format.
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

## **Data Augmentation**

As samples of data for classes 3,4 & 5 are quite low in percentage. We are only applying data augmentation on these classes.

### **Extracting subsets of data classes**

In [ ]:
train_label_5 = train_dataset.filter(lambda example: example["label"] == 5)
train_label_4 = train_dataset.filter(lambda example: example["label"] == 4)
train_label_3 = train_dataset.filter(lambda example: example["label"] == 3)

Filter:   0%|          | 0/288918 [00:00<?, ? examples/s]

Filter:   0%|          | 0/288918 [00:00<?, ? examples/s]

Filter:   0%|          | 0/288918 [00:00<?, ? examples/s]

In [7]:
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/zs07752/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

### **Loading Data Augmentation Models**

We are utilizing four data augmentation techniques:

- Back Translation
- Antonym
- Insert
- Delete

In [8]:
aug_bt = naw.BackTranslationAug(
    from_model_name="Helsinki-NLP/opus-mt-nl-en",  # Dutch to English
    to_model_name="Helsinki-NLP/opus-mt-en-nl"    # English to Dutch
)

In [9]:
aug_antonym = naw.AntonymAug()
aug_random_insert = naw.RandomWordAug(action='substitute')
aug_random_delete = naw.RandomWordAug(action="delete")

### **Function to apply data augmentation**

In [10]:
# Function to augment data
def augment_data(example):
  # BT
  augmented_text_backtranslation = aug_bt.augment(example["text"])

  augmented_text_antonym = aug_antonym.augment(example["text"])
  augmented_text_insert = aug_random_insert.augment(example["text"])
  augmented_text_delete = aug_random_delete.augment(example["text"])

  return {
    "aug_text_bt": augmented_text_backtranslation,
    "aug_text_antonym": augmented_text_antonym,
    "aug_text_insert": augmented_text_insert,
    "aug_text_delete": augmented_text_delete
  }

In [ ]:
augmented_label_5 = train_label_5.map(augment_data)
augmented_label_3 = train_label_3.map(augment_data)
augmented_label_4 = train_label_4.map(augment_data)

Map:   0%|          | 0/10740 [00:00<?, ? examples/s]

Map:   0%|          | 0/14910 [00:00<?, ? examples/s]

### **Merging augmented data from different techniques used**

In [ ]:
# Create a new dataset with the augmented texts
def create_augmented_dataset(augmented_data, label):

    data_list = []
    for i in range(len(augmented_data)):
      data_list.append({
          "text": augmented_data[i]["aug_text_bt"][0],  # augmented text
          "label": label # same lable
      })
      data_list.append({
        "text": augmented_data[i]["aug_text_antonym"][0],  # augmented text
        "label": label  # same lable
      })
      data_list.append({
        "text": augmented_data[i]["aug_text_insert"][0],  # augmented text
        "label": label # same lable
      })
      data_list.append({
        "text": augmented_data[i]["aug_text_delete"][0],  # augmented text
        "label": label # same lable
      })

    return Dataset.from_list(data_list)

In [ ]:
augmented_dataset_4 = create_augmented_dataset(augmented_label_4, 4)
augmented_dataset_4.to_json("augmented_data_4.jsonl", orient="records", lines=True)

Creating json from Arrow format:   0%|          | 0/60 [00:00<?, ?ba/s]

80036383

In [16]:
augmented_dataset_3 = create_augmented_dataset(augmented_label_3, 3)
augmented_dataset_3.to_json("augmented_data_3.jsonl", orient="records", lines=True)

Creating json from Arrow format:   0%|          | 0/43 [00:00<?, ?ba/s]

50869022

In [ ]:
augmented_dataset_5 = create_augmented_dataset(augmented_label_5, 5)
augmented_dataset_5.to_json("augmented_data_5.jsonl", orient="records", lines=True)

### **Merging augmented data for all three classes with the original dataset**

In [ ]:
train_dataset_aug = concatenate_datasets([train_dataset, augmented_dataset_4, augmented_dataset_3, augmented_dataset_5])

In [32]:
train_dataset_aug

Dataset({
    features: ['text', 'label'],
    num_rows: 396990
})

### **Class Distribution of Original Dataset**

In [33]:
# Access the 'label' column directly from the Hugging Face dataset
label_counts = Counter(train_dataset['label'])

# Print the class distribution
print("Class distribution in train dataset:")
for label, count in sorted(label_counts.items()):
    print(f"Label {label}: {count} samples")

Class distribution in train dataset:
Label 0: 75270 samples
Label 1: 95398 samples
Label 2: 91232 samples
Label 3: 10740 samples
Label 4: 14910 samples
Label 5: 1368 samples


### **Class Distribution of Updated Dataset**

In [34]:
# Access the 'label' column directly from the Hugging Face dataset
label_counts = Counter(train_dataset_aug['label'])

# Print the class distribution
print("Class distribution in train dataset after augmentation:")
for label, count in sorted(label_counts.items()):
    print(f"Label {label}: {count} samples")

Class distribution in train dataset after augmentation:
Label 0: 75270 samples
Label 1: 95398 samples
Label 2: 91232 samples
Label 3: 53700 samples
Label 4: 74550 samples
Label 5: 6840 samples


### **Converting Updated Dataset to jsonl**

In [35]:
train_dataset_aug.to_json("augmented_data_full.jsonl", orient="records", lines=True)

Creating json from Arrow format:   0%|          | 0/397 [00:00<?, ?ba/s]

764028133

## **Finetuning & Evaluating the Model on our Update Dataset (With Augmentation)**

Here we are using an evaluation script `baseline_st2.py`.

The parameters we are passing to the script are:

- `train file path`
- `val file path`
- `model name`
- `prediction file path`
- `test file path` (Currently it is just a placeholder as we have not recieved the Test Dataset)

In [ ]:
!python baseline_st2.py --train_file_path "augmented_data_full.jsonl" --dev_file_path "/kaggle/working/st2data/val.jsonl" --test_file_path "/kaggle/working/st2data/val.jsonl" --model bert-base-uncased --prediction_file_path results/subtask2/data_Aug_exp1.csv

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|███████████████████████| 72661/72661 [01:09<00:00, 1042.13 examples/s]
/home/zs07752/anaconda3/envs/llm/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/zs07752/Voight-Kampff-Nexus-Interrogators/baseline.py:74: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
{'loss': 0.6885, 'grad_norm': 15.07113265991211, 'learning_rate': 1.9596969208447528e-05, 'epoch': 0.02}
{'loss': 0.398, 'grad_norm': 5.74927282333374, 'learn